In [1]:
from datascience import *
import numpy as np
from math import *
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

## Lesson 32: Likelihood Ratio Tests
### C2C Jordan Armstrong
#### Documentation: None.

Last time, we introduced Likelihood Ratio tests. Recall that the point of a likelihood ratio test is to compare the likelihood function under a hypothesized value of the parameter with the liklihood function at its maximum. Instead of looking at the ratio $\Lambda$ itself, we often consider $-2\log \Lambda$ instead, since it has a handy distribution. 

### Example 1: Exponential Distribution

Suppose $X_1,X_2,...,X_n$ is an iid sequence of random variables from the exponential distribution with unknown parameter $\lambda$. Recall that the maximum likelihood estimate of $\lambda$ is $1\over\bar{X}$. We collect a random sample of size 20 and want to test the hypothesis $H_0: \lambda = 3$ vs $H_1: \lambda \neq 3$. Using the data in the python box below, conduct a likelihood ratio test on this hypothesis.  

In [2]:
my_data=np.array([0.18,0.277,0.105,0.126,0.225,0.026,0.123,0.423,0.006,0.281,0.050,0.692,0.105,0.275,0.346,0.079,0.045,0.222,0.063,0.281])

In [5]:
# Maximum Likelihood estimate of parameter
xbar = np.mean(my_data)
lam_ML = 1/xbar

# Likelihood ratio test - uses f(x,lambda) for exponential distribution
L_3 = (3**20)*exp(-3*20*xbar)
L_ML = (lam_ML**20)*exp(-lam_ML*20*xbar)
Lambda = L_3/L_ML
Lambda

0.09445694279678146

In [6]:
test_stat = -2*np.log(Lambda)
test_stat

4.719222360188461

In [7]:
p_val = np.round(1-stats.chi2.cdf(test_stat,1),5)
p_val

0.02983

Since the ratio test produces a value that is close to zero, the estimate for $\lambda$ under $H_0$ is unlikely to be true. 

#### Power

Suppose that the true value of $\lambda$ is 5. Let's determine the power of this test. Let $n=20$. Then determine the power if $n=50$. Remember, power is the probability of correctly rejecting the null hypothesis. 

First, find what value of $-2 \log \Lambda$ would lead you to reject $H_0$. This is sometimes called the critical value. 

In [8]:
crit = stats.chi2.ppf(0.95,1)
crit

3.841458820694124

In [9]:
def test_stat(data,null,n):
    lam_ML = 1/np.mean(data)
    return -2*log((np.mean(data)**n)*(null**n)*e**(-null*sum(data)+n))

Next, obtain the power. Obtain a sample of size 20 from the true population and obtain the value of $-2\log \Lambda$ for this sample. Repeat many times and determine how often you reject the null hypothesis. 

In [10]:
ts = []
for i in np.arange(10**4):
    x = stats.expon.rvs(scale=1/5,size=20) # random sample
    ts = np.append(ts,test_stat(x,3,20))

# Comparing test statistic to the critical value
power20 = np.mean(ts>=crit) 
power20

0.6002

Repeat for a sample size of 50. What do you expect to happen to power? 

In [11]:
ts = []
for i in np.arange(10**4):
    x = stats.expon.rvs(scale=1/5,size=50) # random sample
    ts = np.append(ts,test_stat(x,3,50))

# Comparing test statistic to the critical value
power50 = np.mean(ts>=crit) 
power50

0.9517

This makes sense as for a larger sample size we are decrasing the probability of a Type 1 Error and thus the power should be closer to 1.

### A Different Test

We've explored hypothesis tests in this class before. Taking advantage of our computing power, we don't have to rely on test statistics with asymptotic distributions. Let's conduct a more direct hypothesis test using simulation. Recall:

$$
H_0: \lambda = 3
$$

$$
H_1: \lambda \neq 3
$$

Pick a different test statistic. Obtain an empirical distribution of that test statistic under $H_0$. Next, find the $p$-value by determining how often this test statistic is at or further away from the test statistic derived from the sample. Remember that this is a two-sided test. 

In [12]:
ts2 = []
for i in np.arange(10**4):
    ts2 = np.append(ts2,np.mean(stats.expon.rvs(scale=1/3,size=20))) # new x-bar

# Two-sided test
2*np.mean(ts2<=xbar)

0.0342

How did the $p$-value compare to the LRT $p$-value? I wonder how the power of this test compares to our LRT. 

#### Power

Let's figure out the power of this test. First, determine for what values of the test statistic would lead us to reject $H_0$. These values can be referred to as your rejection region. 

In [13]:
crit_low = percentile(2.5,ts2)
crit_high = percentile(97.5,ts2)
[crit_low,crit_high]

[0.2047097992505325, 0.491698034556714]

Now, determine the power of this test. Like in the LRT case, obtain a sample of size 20 and obtain the test statistic. Repeat many times and see how often your test statistic is in your rejection region. 

In [14]:
ts2 = []
for i in np.arange(10**4):
    ts2 = np.append(ts2,np.mean(stats.expon.rvs(scale=1/5,size=20)))

# Comparing test statistic 2 to the critical values
power20 = np.mean(ts2<=crit_low) + np.mean(ts2>=crit_high)
power20

0.5661

Repeat for a sample size of 50. Note that you will have to obtain new critical values in order to do this.  

In [15]:
ts2 = []
for i in np.arange(10**4):
    ts2 = np.append(ts2,np.mean(stats.expon.rvs(scale=1/3,size=50)))

# Recalculate critical values for n = 50
crit_low = percentile(2.5,ts2)
crit_high = percentile(97.5,ts2)

ts2 = []
for i in np.arange(10**4):
    ts2 = np.append(ts2,np.mean(stats.expon.rvs(scale=1/5,size=50)))
    
# Comparing test statistic 2 to the critical values
power50 = np.mean(ts2<=crit_low) + np.mean(ts2>=crit_high)
power50

0.9429